In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import numpy as np

import warnings
warnings.simplefilter("ignore", UserWarning)

%matplotlib inline

In [3]:
from PIL import Image, ImageColor, ImageFont, ImageDraw, ImageFilter

In [25]:
# import DL stuff
import torch
import torch.nn as nn

import torch.functional as F
from torch.autograd import Variable as V
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tensorflow as tf

slim = tf.contrib.slim

In [ ]:
from helpers import Converter, Resize, Normalize, 

In [5]:
print('tf:', tf.__version__, '\n\r', 'torch:', torch.__version__)

tf: 1.12.0 
 torch: 1.0.0


In [6]:
import pandas as pd

In [11]:
data = pd.read_csv('./data/data.csv', sep=';', header=None)

In [17]:
len(data[0])

100

In [20]:
class DataStream(Dataset):
    def __init__(self, data, transform=None):
        self.data = pd.read_csv('./data/data.csv', sep=';', header=None)
        self.imgs = self.data[0]
        self.labels = self.data[1]
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        x = io.imread(self.imgs[idx])
        y = self.labels[idx]
        sample = {'img' : x, 'label' : y}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [23]:
dr = DataStream(data)

In [24]:
sample = next(dr)

TypeError: 'DataStream' object is not an iterator

In [107]:
H = 32
W = 280
C = 1
net = CRNN(C, nc=32, nh=128)
net.apply(weights_init)
X = V(torch.randn(1, C, H, W))
Y_ = net(X)

torch.Size([1, 512, 1, 71])
torch.Size([71, 1, 32])


In [ ]:
opt = nn.CTCLoss(

In [200]:
16 * 7

112

In [48]:
class AttrDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    
def define_config():
    config = AttrDict()
    config.n_classes = 34
    config.lstm_size = 256
    config.width = 280
    config.height = 32
    return config

In [49]:
cfg

{'n_classes': 34, 'lstm_size': 256}

In [59]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell

import numpy as np

tf.enable_eager_execution()

slim = tf.contrib.slim
cfg = define_config()

def cnn(inputs, scope='vgg', is_training=True):
    batch_norm_params = {'is_training': is_training}
    with slim.arg_scope([slim.conv2d], 
                        normalizer_fn=slim.batch_norm, 
                        normalizer_params=batch_norm_params,):
        with slim.arg_scope([slim.batch_norm], **batch_norm_params):
            net = slim.repeat(
                inputs, 1, 
                slim.conv2d, 
                64, [3, 3], 
                scope='conv1',
            )
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 1, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2', padding='SAME')
            net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], stride=[2, 1], scope='pool3', padding='SAME')
            net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], stride=[2, 1], scope='pool4', padding='SAME')
            net = slim.repeat(net, 1, slim.conv2d, 512, [3, 3], scope='conv5')
            return net


def crnn(images, is_training=True):
    dropout_keep_prob = 0.7 if is_training else 1.0
    cnn_net = cnn(images, is_training=is_training)
    with tf.variable_scope('Reshaping_cnn'):
        shape = cnn_net.get_shape().as_list()  # [batch, height, width, features]
        transposed = tf.transpose(cnn_net, perm=[0, 2, 1, 3],
                                  name='transposed')  # [batch, width, height, features]
        conv_reshaped = tf.reshape(transposed, [shape[0], -1, shape[1] * shape[3]],
                                   name='reshaped')  # [batch, width, height x features]

    list_n_hidden = [cfg.lstm_size, cfg.lstm_size]

    with tf.name_scope('deep_bidirectional_lstm'):
        # Forward direction cells
        fw_cell_list = [BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]
        # Backward direction cells
        bw_cell_list = [BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]

        lstm_net, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(fw_cell_list,
                                                                        bw_cell_list,
                                                                        conv_reshaped,
                                                                        dtype=tf.float32
                                                                        )
        # Dropout layer
        lstm_net = tf.nn.dropout(lstm_net, keep_prob=dropout_keep_prob)
        #logging.info('after lstm shape: %s' % lstm_net.shape)

    with tf.variable_scope('fully_connected'):
        shape = lstm_net.get_shape().as_list()  # [batch, width, 2*n_hidden]
        fc_out = slim.layers.linear(lstm_net, cfg.n_classes)  # [batch x width, n_class]
        #logging.info('fc_out shape: %s' % fc_out.shape)

        lstm_out = tf.reshape(fc_out, [shape[0], -1, cfg.n_classes],
                              name='lstm_out')  # [batch, width, n_classes]
        #logging.info('lstm_out shape: %s' % lstm_out.shape)

        # Swap batch and time axis
        logprob = tf.transpose(lstm_out, [1, 0, 2], name='transpose_time_major')  # [width(time), batch, n_classes]

        return logprob


def create_loss(sparse_code_target, logprob, seq_len_inputs):
    with tf.control_dependencies(
            [tf.less_equal(sparse_code_target.dense_shape[1], tf.reduce_max(tf.cast(seq_len_inputs, tf.int64)))]):
        loss_ctc = tf.nn.ctc_loss(labels=sparse_code_target,
                                  inputs=logprob,
                                  sequence_length=tf.cast(seq_len_inputs, tf.int32),
                                  ignore_longer_outputs_than_inputs=True,
                                 )
        loss_ctc = tf.reduce_mean(loss_ctc)
    return loss_ctc

In [60]:
ret = crnn(np.random.rand(1,224,64,3).astype(np.float32))

W0211 16:57:17.981990 140354946066176 tf_logging.py:161] <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fa648325710>: Note that this cell is not optimized for performance. Please use tf.contrib.cudnn_rnn.CudnnLSTM for better performance on GPU.
W0211 16:57:17.984064 140354946066176 tf_logging.py:161] <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fa648325940>: Note that this cell is not optimized for performance. Please use tf.contrib.cudnn_rnn.CudnnLSTM for better performance on GPU.
W0211 16:57:17.985587 140354946066176 tf_logging.py:161] <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fa648325208>: Note that this cell is not optimized for performance. Please use tf.contrib.cudnn_rnn.CudnnLSTM for better performance on GPU.
W0211 16:57:17.987194 140354946066176 tf_logging.py:161] <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fa6483255c0>: Note that this cell is not optimized for performance. Please use tf.contrib.cud

In [61]:
ret.shape

TensorShape([Dimension(16), Dimension(1), Dimension(34)])

In [ ]:
def encode(text):
    if isinstance(text, str):
        te